In [1]:
# Standard library imports
import asyncio
import os
import sqlite3
from io import StringIO

# Third-party imports
import nest_asyncio
import pandas as pd
from bs4 import BeautifulSoup
from scrapling import StealthyFetcher

# scrape one year

In [2]:
async def scrape_year(player_id, year):
    # The URL pattern for pitching game logs
    url = f"https://www.baseball-reference.com/players/gl.fcgi?id={player_id}&t=p&year={year}"
    page = await StealthyFetcher().async_fetch(url)  # the async version of fetch
    soup = BeautifulSoup(page.html_content, 'html.parser')
    #extract the player name from the title
    player_name = ' '.join(soup.find('title').text.split()[:2]) if soup.find('title') else None
    game_log_table = soup.find('table', id='pitching_gamelogs')
    df = pd.read_html(StringIO(str(game_log_table)))[0] if game_log_table else []
    if not isinstance(df, list) and not df.empty:
        df['year'] = year
        df['player_id'] = player_id
        df['name'] = player_name
    return df, player_name

# scrape all years for 1 guy

In [3]:
async def player_scrape(player_id):
    """Scrape and combine pitching data for 2021-2024 for a player."""
    years = [2021, 2022, 2023, 2024]  # Fixed years to scrape
    tasks = [scrape_year(player_id, year) for year in years] # run all the years at once
    results = await asyncio.gather(*tasks)
    # Combine dataframes, filtering out empty ones
    dfs = [df for df, _ in results if not isinstance(df, list) and not df.empty]
    combined_df = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()
    return combined_df

## Clean data

In [4]:
def cleanse_pitcher_game_logs(df):
    rename_map = {
        'Rk': 'season_game_num',
        'Gcar': 'career_game_num',
        'Gtm': 'team_game_num',
        'Date': 'date',
        'Tm': 'team_id',
        'Unnamed: 5': 'road_indicator',
        'Opp': 'opp_id',
        'Rslt': 'game_result',
        'Inngs': 'innings',
        'DR': 'days_rest',
        'IP': 'ip',
        'H': 'h',
        'R': 'r',
        'ER': 'er',
        'BB': 'bb',
        'SO': 'so',
        'HR': 'hr',
        'HBP': 'hbp',
        'ERA': 'era',
        'FIP': 'fip',
        'BF': 'batters_faced',
        'Pit': 'pitches',
        'Str': 'strikes_total',
        'StL': 'strikes_looking',
        'StS': 'strikes_swinging',
        'GB': 'ground_balls',
        'FB': 'flyballs',
        'LD': 'line_drives',
        'PU': 'pop_ups',
        'GSc': 'game_score',
        'SB': 'sb',
        'CS': 'cs',
        'PO': 'pickoffs',
        'AB': 'ab',
        '2B': '2b',
        '3B': '3b',
        'IBB': 'ibb',
        'GDP': 'gidp',
        'SF': 'sf',
        'ROE': 'roe',
        'aLI': 'avg_leverage_index',
        'WPA': 'win_prob_added',
        'acLI': 'adjusted_cli_avg',
        'cWPA': 'champ_win_prob_added',
        'RE24': 'base_out_run_saved'
    }
    #drop un needed and apply column mapping
    df = df.drop(columns=['Dec','IR','IS','Unk','DFS(DK)','DFS(FD)', 'Entered', 'Exited','Rslt','Inngs'], 
                                    errors='ignore').rename(columns=rename_map)
    # Convert road_indicator to dummy variables and filter invalid team_id rows
    df = pd.concat([df.drop('road_indicator', axis=1), 
                            pd.get_dummies(df['road_indicator'], prefix='road')], axis=1)
    df = df[~((df['team_id'].isna()) | (df['team_id'] == "Tm"))]
    # Ensure player_id, name, and year are first three columns
    desired_order = ['player_id', 'name', 'year'] + [col for col in df.columns if col not in ['player_id', 'name', 'year']]
    df = df[desired_order]
    return df

# Store Data

In [5]:
def save_pitcher_logs(df, db_path="baseball.db"):
    db_path = os.path.abspath(db_path)
    
    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(db_path) or '.', exist_ok=True)
    # Create connection to SQLite database
    conn = sqlite3.connect(db_path)
    
    #save gamelogs
    df.to_sql("pitching_gamelogs", conn, if_exists='append', index=False)
    
    #save player info
    player_info = df[['player_id', 'name']].drop_duplicates()
    player_info.to_sql("scraped_players", conn, if_exists='append', index=False)
    
    # Close connection
    conn.close()
    
    print(f"{df['name'].iloc[0]} finished")


# Put it all together


In [6]:
async def scrape_all_players(filepath):
    test_ids = (pd.read_csv(filepath))["player_id"]
    for player_id in test_ids:
        print(player_id)
        df = await player_scrape(player_id)
        df = cleanse_pitcher_game_logs(df)
        save_pitcher_logs(df)


# Run it on csv file

In [ ]:
# Apply nest_asyncio to allow asyncio to run in Jupyter Notebook
nest_asyncio.apply()

# Filepath to your CSV file (update this if needed)
csv_filepath = "test_ids.csv"  # Adjust if it's in a different directory

# Run the async function in the event loop
asyncio.run(scrape_all_players(csv_filepath))
